Imports and set pandas dataframe display options:

In [1]:
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import sklearn
import matplotlib.pyplot as plt
import re
from IPython.display import display
from html.parser import HTMLParser

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

Load the scraped data and have a cursory look:

In [2]:
df = pd.read_csv("porvoo_listings.csv")
display(df.head())
display(df.describe())

,Unnamed: 0,Sijainti,Kaupunginosa,Kohdenumero,Kerros,Asuinpinta-ala,Tontin pinta-ala,Kokonaispinta-ala,Pinta-alojen lisätiedot,Huoneiston kokoonpano,Huoneita,Kunto,Kunnon lisätiedot,Lisätietoa vapautumisesta,Keittiön varusteet,Kylpyhuoneen varusteet,Tulevat remontit,Tehdyt remontit,Asunnossa sauna,Saunan lisätiedot,Asumistyyppi,Palvelut,Lisätiedot,Kohde on,Linkit,Velaton hinta,Myyntihinta,Neliöhinta,Hoitovastike,Yhtiövastike,Vesimaksu,Vesimaksun lisätiedot,Muut kustannukset,Uudiskohde,Taloyhtiön nimi,Rakennuksen tyyppi,Rakennusvuosi,Huoneistojen lukumäärä,Kerroksia,Hissi,Rakennusmateriaali,Kattomateriaali,Kattotyyppi,Energialuokka,Energiatodistus,Tontin koko,Tontin omistus,Kiinteistönhoito,Isännöinti,Kaavoitustiedot,Kaavatilanne,Liikenneyhteydet,Lämmitys,Lisätietoja talosta,Pysäköintitilan kuvaus,Yhteiset tilat,Muut tilat,Pintamateriaalit,Lisätietoja makuuhuoneen varusteista,Olohuoneen varusteet,Kuvaus,Parveke,Parvekkeen lisätiedot,Säilytystilat,Näkymät,Vuokrattu,Pysäköintimaksu,Pinta-alan lähde,Julkisivuremontti,Postitoimipaikka,Lämmityskustannukset,Rakennuksen käyttöönottovuosi,Taloyhtiössä on sauna,Perustus,Kunnallistekniikka,Ilmanvaihto,Rakennusoikeus,Maasto,Lainaosuuden maksu,Velkaosuus,Rahoitusvastike,Saunan kustannukset,Kiinteistön antennijärjestelmä,Muut materiaalit,Kerroksien lukumäärä,Terassin kuvaus,Ikkunoiden suunta,Rannan (vesistön) tyyppi,Rannan omistus,Kiinteistötunnus,Tietoliikennepalvelut,Keskimääräinen sähkönkulutus,Kiinteistövero,Tilan nimi,Tontin kuvaus,Rasitteet ja oikeudet,Viemäröinti,Käyttövesi,Kuntotutkimus,Liiketilojen pinta-ala,Liikehuoneistojen lukumäärä,Lunastuspykälä,Lisätietoa taloyhtiöstä,Kattoremontti,Putkiremontti,Maanvuokraaja,Tontin vuokra,Uudiskohteen tila,Ajo-ohjeet,Kiinnitykset,Tontin vuokra päättyy,Lisätietoa rakennusoikeudesta,Keittiö,Kylpyhuoneen remontit,Keittiön remontit,Muut remontit,Osakeluettelo on siirretty huoneistojärjestelmään,Sähköinen omistajamerkintä,TV-palvelut,Vuokratulo,Liikehuoneistojen pinta-ala,Lisätietoa hissistä,Lisätietoa taloyhtiön saunasta,Kiinteistön laajakaistayhteys,Harrastusmahdollisuudet,Rungon rakennustapa,Keittiön lattia,Keittiön seinä,Makuuhuoneen lattia,Makuuhuoneen seinät,Kylpyhuoneen lattia,Kylpyhuoneen seinät,Olohuoneen lattia,Olohuoneen seinät,Ominaisuudet,Vesi ja jätevesi,Vapautuu,Ensiesittelyssä,Tontin vuokravastike,Vuokratontin lunastusosuus,Kunnan numero,Kerroksen lisätiedot,Autopaikan kustannukset,Rakennusvuoden lisätiedot,Pihan ilmansuunnat,Puhtaanapito,Ilmansuunnat,Tiedustelut,Vuokra-aika,Muut rakennukset,Pihan kuvaus,Kerrosala,Uusin kirjallinen tarjous,Rannan kuvaus,Tiemaksut,Sähkömaksu,Soveltuvuus liikuntaesteisille,Taloyhtiön tilat ja tuotot,Korjausvastike,Saunamaksu,Valmistumisaika,Rannan pituus,Varaustilanne,Rannan suunta,Alueen kuvaus,Vesistö
0,0,"<a href=""/myytavat-asunnot?locations=%5B%22Puo...","<a href=""/myytavat-asunnot/porvoo/j%C3%A4rnb%C...",21875705,1 / 3,"102,5 m²",4 395 m²,"102,5 m²","yhtiöjärjestyksen mukainen, isännöitsijäntodis...",4h+k+s+ 2 pihaterassia,4.0,Hyvä,Siistissä kunnossa oleva muuttovalmis asunto.,sopimuksen mukaan,Upea ja uniikki puusepän valmistama keittiö on...,"Pesutilojen kuvaus: Tilava kylpyhuone, jossa o...",Yhtiön hallituksen 25.1.2021 päivätyn kunnossa...,HUONEISTOSSA: Keittiö on uusittu kodinkoneinee...,Kyllä,"Kuvaus: Oma sauna, jossa on sähkökiuas. .",Omistus,"Palvelut: Sale Huhtinen n. 1,5 km, K-Citymarke...","Muuta kauppaan kuuluvaa: Ilmalämpöpumppu, kii...",Osakehuoneisto,"<ul class=""link-list""><li class=""link-list__it...",378&nbsp;000&nbsp;€,378&nbsp;000&nbsp;€,"3&nbsp;687,80&nbsp;€ / m<sup>2</sup>","417,15&nbsp;€ / kk","417,15&nbsp;€ / kk",17&nbsp;€ / kk,henkilöluvun mukaan,Muut maksut: Internet/kuituverkko käyttökorvau...,Ei,Asunto Oy Porvoon Puolukkapolku 2 - Bostads Ab...,Kerrostalo,2009.0,18.0,3.0,Kyllä,"tiili, betoni",pelti,harjakatto,E2007 ET-luku: 194. Todistus annettu 31.5.2013.,Kyllä,4 395 m²,Oma,huoltoyhtiö. Kiinteistönhoidon lisätiedot: Por...,"OP Isännöinti Itä-Uusimaa Oy, Tero Korj

,Unnamed: 0,Huoneita,Rakennusvuosi,Huoneistojen lukumäärä,Kerroksia,Rakennuksen käyttöönottovuosi,Kerroksien lukumäärä,Liikehuoneistojen lukumäärä,Putkiremontti,Kunnan numero
count,299.000000,296.000000,287.000000,203.000000,224.000000,54.000000,19.000000,54.000000,2.000000,12.0
mean,149.000000,3.155405,1997.379791,36.280788,3.035714,1984.925926,2.105263,1.370370,2013.000000,638.0
std,86.458082,1.610191,34.701202,40.500152,1.781462,26.242592,0.567131,2.349599,8.485281,0.0
min,0.000000,0.000000,1804.000000,2.000000,1.000000,1899.000000,1.000000,0.000000,2007.000000,638.0
25%,74.500000,2.000000,1976.000000,23.500000,2.000000,1973.250000,2.000000,0.000000,2010.000000,638.0
50%,149.000000,3.000000,2017.000000,32.000000,2.000000,1981.500000,2.000000,1.000000,2013.000000,638.0
75%,223.500000,4.000000,2022.000000,37.500000,4.000000,2009.500000,2.000000,1.000000,2016.000000,638.0
max,298.000000,10.000000,2023.000000,388.000000,8.000000,2022.000000,3.000000,10.000000,2019.000000,638.0


First we tackle the columns with html tags:

In [3]:
#Define MyHTMLParser the functionality of appending found data into a string
class MyHTMLParser(HTMLParser):
    def __init__(self):
      super().__init__()
      self.string = ''
    def handle_data(self, data):
      self.string += data
#This function creates an instance of MyHTMLParser and returns the string made out of the data portions of the html
def parse(html : str) -> str:
    parser = MyHTMLParser()
    parser.feed(html)
    return parser.string

df['Sijainti'] = df['Sijainti'].apply(parse)
df['Myyntihinta'] = df['Myyntihinta'].apply(parse)
#Kaupunginosa column contains a few NaN values
df['Kaupunginosa'] = df['Kaupunginosa'].apply(lambda x: parse(x) if pd.notna(x) else x)
display(df.head())

,Unnamed: 0,Sijainti,Kaupunginosa,Kohdenumero,Kerros,Asuinpinta-ala,Tontin pinta-ala,Kokonaispinta-ala,Pinta-alojen lisätiedot,Huoneiston kokoonpano,Huoneita,Kunto,Kunnon lisätiedot,Lisätietoa vapautumisesta,Keittiön varusteet,Kylpyhuoneen varusteet,Tulevat remontit,Tehdyt remontit,Asunnossa sauna,Saunan lisätiedot,Asumistyyppi,Palvelut,Lisätiedot,Kohde on,Linkit,Velaton hinta,Myyntihinta,Neliöhinta,Hoitovastike,Yhtiövastike,Vesimaksu,Vesimaksun lisätiedot,Muut kustannukset,Uudiskohde,Taloyhtiön nimi,Rakennuksen tyyppi,Rakennusvuosi,Huoneistojen lukumäärä,Kerroksia,Hissi,Rakennusmateriaali,Kattomateriaali,Kattotyyppi,Energialuokka,Energiatodistus,Tontin koko,Tontin omistus,Kiinteistönhoito,Isännöinti,Kaavoitustiedot,Kaavatilanne,Liikenneyhteydet,Lämmitys,Lisätietoja talosta,Pysäköintitilan kuvaus,Yhteiset tilat,Muut tilat,Pintamateriaalit,Lisätietoja makuuhuoneen varusteista,Olohuoneen varusteet,Kuvaus,Parveke,Parvekkeen lisätiedot,Säilytystilat,Näkymät,Vuokrattu,Pysäköintimaksu,Pinta-alan lähde,Julkisivuremontti,Postitoimipaikka,Lämmityskustannukset,Rakennuksen käyttöönottovuosi,Taloyhtiössä on sauna,Perustus,Kunnallistekniikka,Ilmanvaihto,Rakennusoikeus,Maasto,Lainaosuuden maksu,Velkaosuus,Rahoitusvastike,Saunan kustannukset,Kiinteistön antennijärjestelmä,Muut materiaalit,Kerroksien lukumäärä,Terassin kuvaus,Ikkunoiden suunta,Rannan (vesistön) tyyppi,Rannan omistus,Kiinteistötunnus,Tietoliikennepalvelut,Keskimääräinen sähkönkulutus,Kiinteistövero,Tilan nimi,Tontin kuvaus,Rasitteet ja oikeudet,Viemäröinti,Käyttövesi,Kuntotutkimus,Liiketilojen pinta-ala,Liikehuoneistojen lukumäärä,Lunastuspykälä,Lisätietoa taloyhtiöstä,Kattoremontti,Putkiremontti,Maanvuokraaja,Tontin vuokra,Uudiskohteen tila,Ajo-ohjeet,Kiinnitykset,Tontin vuokra päättyy,Lisätietoa rakennusoikeudesta,Keittiö,Kylpyhuoneen remontit,Keittiön remontit,Muut remontit,Osakeluettelo on siirretty huoneistojärjestelmään,Sähköinen omistajamerkintä,TV-palvelut,Vuokratulo,Liikehuoneistojen pinta-ala,Lisätietoa hissistä,Lisätietoa taloyhtiön saunasta,Kiinteistön laajakaistayhteys,Harrastusmahdollisuudet,Rungon rakennustapa,Keittiön lattia,Keittiön seinä,Makuuhuoneen lattia,Makuuhuoneen seinät,Kylpyhuoneen lattia,Kylpyhuoneen seinät,Olohuoneen lattia,Olohuoneen seinät,Ominaisuudet,Vesi ja jätevesi,Vapautuu,Ensiesittelyssä,Tontin vuokravastike,Vuokratontin lunastusosuus,Kunnan numero,Kerroksen lisätiedot,Autopaikan kustannukset,Rakennusvuoden lisätiedot,Pihan ilmansuunnat,Puhtaanapito,Ilmansuunnat,Tiedustelut,Vuokra-aika,Muut rakennukset,Pihan kuvaus,Kerrosala,Uusin kirjallinen tarjous,Rannan kuvaus,Tiemaksut,Sähkömaksu,Soveltuvuus liikuntaesteisille,Taloyhtiön tilat ja tuotot,Korjausvastike,Saunamaksu,Valmistumisaika,Rannan pituus,Varaustilanne,Rannan suunta,Alueen kuvaus,Vesistö
0,0,"Puolukkapolku 2 A, 06100 Porvoo",Järnböle,21875705,1 / 3,"102,5 m²",4 395 m²,"102,5 m²","yhtiöjärjestyksen mukainen, isännöitsijäntodis...",4h+k+s+ 2 pihaterassia,4.0,Hyvä,Siistissä kunnossa oleva muuttovalmis asunto.,sopimuksen mukaan,Upea ja uniikki puusepän valmistama keittiö on...,"Pesutilojen kuvaus: Tilava kylpyhuone, jossa o...",Yhtiön hallituksen 25.1.2021 päivätyn kunnossa...,HUONEISTOSSA: Keittiö on uusittu kodinkoneinee...,Kyllä,"Kuvaus: Oma sauna, jossa on sähkökiuas. .",Omistus,"Palvelut: Sale Huhtinen n. 1,5 km, K-Citymarke...","Muuta kauppaan kuuluvaa: Ilmalämpöpumppu, kii...",Osakehuoneisto,"<ul class=""link-list""><li class=""link-list__it...",378&nbsp;000&nbsp;€,378 000 €,"3&nbsp;687,80&nbsp;€ / m<sup>2</sup>","417,15&nbsp;€ / kk","417,15&nbsp;€ / kk",17&nbsp;€ / kk,henkilöluvun mukaan,Muut maksut: Internet/kuituverkko käyttökorvau...,Ei,Asunto Oy Porvoon Puolukkapolku 2 - Bostads Ab...,Kerrostalo,2009.0,18.0,3.0,Kyllä,"tiili, betoni",pelti,harjakatto,E2007 ET-luku: 194. Todistus annettu 31.5.2013.,Kyllä,4 395 m²,Oma,huoltoyhtiö. Kiinteistönhoidon lisätiedot: Por...,"OP Isännöinti Itä-Uusimaa Oy, Tero Korjula, pu...","Porvoon kaupunki, (019) 520 211",Asemakaava,Linja-autoasema 

Clean column values related to pricing:

In [4]:
#Columns containing values related to pricing
hinnat = ["Velaton hinta", "Myyntihinta", "Neliöhinta", "Hoitovastike", "Yhtiövastike", "Vesimaksu", "Velkaosuus", "Rahoitusvastike", "Saunan kustannukset", "Autopaikan kustannukset", "Kiinteistövero"]
pinta_alat = ["Asuinpinta-ala", "Tontin pinta-ala", "Kokonaispinta-ala", "Tontin koko", "Rakennusoikeus", "Liiketilojen pinta-ala"]
#Strip string &nbsp; and \xa0 from all column values 
df = df.apply(lambda hinta: hinta.str.replace('&nbsp;', '') if hinta.name in hinnat else hinta)
df = df.apply(lambda hinta: hinta.str.replace("\\xa0","") if hinta.name in hinnat else hinta)
df = df.apply(lambda hinta: hinta.str.replace("noin","") if hinta.name in hinnat else hinta)

#Replace <sup>2</sup> with latex superscript $^2$
df['Neliöhinta'] = df['Neliöhinta'].str.replace('<sup>2</sup>', '$^2$')
#Clean area related values
df[pinta_alat] = df[pinta_alat].apply(lambda x: x.str.replace("m²", ""))
#Check cleaned columns
#Remove units from values and convert them to floats
sep = '€'
for hinta in hinnat:
#     df[hinta] = df[hinta].apply(lambda x: float(x.split(sep, 1)[0].replace(',', '.')) if pd.notna(x) and x != 'Kysy hintaa' else np.NaN) 
    df[hinta] = df[hinta].apply(lambda x: float(re.split(r'[(\u20AC)e]', x)[0].replace(',', '.')) if pd.notna(x) and x != 'Kysy hintaa' and x != "Ei vielä määrätty" and x != "Uusimaa" and x != "Finland" else np.NaN)
display(df.head())
display(df.describe())

C:\Users\Rantimo\AppData\Local\Temp/ipykernel_20804/758136814.py:6: FutureWarning: The default value of regex will change from True to False in a future version.
  df = df.apply(lambda hinta: hinta.str.replace("\\xa0","") if hinta.name in hinnat else hinta)


,Unnamed: 0,Sijainti,Kaupunginosa,Kohdenumero,Kerros,Asuinpinta-ala,Tontin pinta-ala,Kokonaispinta-ala,Pinta-alojen lisätiedot,Huoneiston kokoonpano,Huoneita,Kunto,Kunnon lisätiedot,Lisätietoa vapautumisesta,Keittiön varusteet,Kylpyhuoneen varusteet,Tulevat remontit,Tehdyt remontit,Asunnossa sauna,Saunan lisätiedot,Asumistyyppi,Palvelut,Lisätiedot,Kohde on,Linkit,Velaton hinta,Myyntihinta,Neliöhinta,Hoitovastike,Yhtiövastike,Vesimaksu,Vesimaksun lisätiedot,Muut kustannukset,Uudiskohde,Taloyhtiön nimi,Rakennuksen tyyppi,Rakennusvuosi,Huoneistojen lukumäärä,Kerroksia,Hissi,Rakennusmateriaali,Kattomateriaali,Kattotyyppi,Energialuokka,Energiatodistus,Tontin koko,Tontin omistus,Kiinteistönhoito,Isännöinti,Kaavoitustiedot,Kaavatilanne,Liikenneyhteydet,Lämmitys,Lisätietoja talosta,Pysäköintitilan kuvaus,Yhteiset tilat,Muut tilat,Pintamateriaalit,Lisätietoja makuuhuoneen varusteista,Olohuoneen varusteet,Kuvaus,Parveke,Parvekkeen lisätiedot,Säilytystilat,Näkymät,Vuokrattu,Pysäköintimaksu,Pinta-alan lähde,Julkisivuremontti,Postitoimipaikka,Lämmityskustannukset,Rakennuksen käyttöönottovuosi,Taloyhtiössä on sauna,Perustus,Kunnallistekniikka,Ilmanvaihto,Rakennusoikeus,Maasto,Lainaosuuden maksu,Velkaosuus,Rahoitusvastike,Saunan kustannukset,Kiinteistön antennijärjestelmä,Muut materiaalit,Kerroksien lukumäärä,Terassin kuvaus,Ikkunoiden suunta,Rannan (vesistön) tyyppi,Rannan omistus,Kiinteistötunnus,Tietoliikennepalvelut,Keskimääräinen sähkönkulutus,Kiinteistövero,Tilan nimi,Tontin kuvaus,Rasitteet ja oikeudet,Viemäröinti,Käyttövesi,Kuntotutkimus,Liiketilojen pinta-ala,Liikehuoneistojen lukumäärä,Lunastuspykälä,Lisätietoa taloyhtiöstä,Kattoremontti,Putkiremontti,Maanvuokraaja,Tontin vuokra,Uudiskohteen tila,Ajo-ohjeet,Kiinnitykset,Tontin vuokra päättyy,Lisätietoa rakennusoikeudesta,Keittiö,Kylpyhuoneen remontit,Keittiön remontit,Muut remontit,Osakeluettelo on siirretty huoneistojärjestelmään,Sähköinen omistajamerkintä,TV-palvelut,Vuokratulo,Liikehuoneistojen pinta-ala,Lisätietoa hissistä,Lisätietoa taloyhtiön saunasta,Kiinteistön laajakaistayhteys,Harrastusmahdollisuudet,Rungon rakennustapa,Keittiön lattia,Keittiön seinä,Makuuhuoneen lattia,Makuuhuoneen seinät,Kylpyhuoneen lattia,Kylpyhuoneen seinät,Olohuoneen lattia,Olohuoneen seinät,Ominaisuudet,Vesi ja jätevesi,Vapautuu,Ensiesittelyssä,Tontin vuokravastike,Vuokratontin lunastusosuus,Kunnan numero,Kerroksen lisätiedot,Autopaikan kustannukset,Rakennusvuoden lisätiedot,Pihan ilmansuunnat,Puhtaanapito,Ilmansuunnat,Tiedustelut,Vuokra-aika,Muut rakennukset,Pihan kuvaus,Kerrosala,Uusin kirjallinen tarjous,Rannan kuvaus,Tiemaksut,Sähkömaksu,Soveltuvuus liikuntaesteisille,Taloyhtiön tilat ja tuotot,Korjausvastike,Saunamaksu,Valmistumisaika,Rannan pituus,Varaustilanne,Rannan suunta,Alueen kuvaus,Vesistö
0,0,"Puolukkapolku 2 A, 06100 Porvoo",Järnböle,21875705,1 / 3,"102,5",4 395,"102,5","yhtiöjärjestyksen mukainen, isännöitsijäntodis...",4h+k+s+ 2 pihaterassia,4.0,Hyvä,Siistissä kunnossa oleva muuttovalmis asunto.,sopimuksen mukaan,Upea ja uniikki puusepän valmistama keittiö on...,"Pesutilojen kuvaus: Tilava kylpyhuone, jossa o...",Yhtiön hallituksen 25.1.2021 päivätyn kunnossa...,HUONEISTOSSA: Keittiö on uusittu kodinkoneinee...,Kyllä,"Kuvaus: Oma sauna, jossa on sähkökiuas. .",Omistus,"Palvelut: Sale Huhtinen n. 1,5 km, K-Citymarke...","Muuta kauppaan kuuluvaa: Ilmalämpöpumppu, kii...",Osakehuoneisto,"<ul class=""link-list""><li class=""link-list__it...",378000.0,378000.00,3687.80,417.15,417.15,17.0,henkilöluvun mukaan,Muut maksut: Internet/kuituverkko käyttökorvau...,Ei,Asunto Oy Porvoon Puolukkapolku 2 - Bostads Ab...,Kerrostalo,2009.0,18.0,3.0,Kyllä,"tiili, betoni",pelti,harjakatto,E2007 ET-luku: 194. Todistus annettu 31.5.2013.,Kyllä,4 395,Oma,huoltoyhtiö. Kiinteistönhoidon lisätiedot: Por...,"OP Isännöinti Itä-Uusimaa Oy, Tero Korjula, pu...","Porvoon kaupunki, (019) 520 211",Asemakaava,Linja-autoasema n. 2 km.,muu,"<a href=""https://asunnot.oikotie.fi/talo/puolu...",autokatoksia: 16. sähköpis

,Unnamed: 0,Huoneita,Velaton hinta,Myyntihinta,Neliöhinta,Hoitovastike,Yhtiövastike,Vesimaksu,Rakennusvuosi,Huoneistojen lukumäärä,Kerroksia,Rakennuksen käyttöönottovuosi,Velkaosuus,Rahoitusvastike,Saunan kustannukset,Kerroksien lukumäärä,Kiinteistövero,Liikehuoneistojen lukumäärä,Putkiremontti,Kunnan numero,Autopaikan kustannukset
count,299.000000,296.000000,227.000000,2.980000e+02,227.000000,216.000000,210.000000,156.000000,287.000000,203.000000,224.000000,54.000000,98.000000,165.000000,22.000000,19.000000,59.000000,54.000000,2.000000,12.0,13.000000
mean,149.000000,3.155405,237960.123348,2.067497e+05,3867.088899,227.216667,426.417762,20.475385,1997.379791,36.280788,3.035714,1984.925926,105631.241939,243.129333,10.863636,2.105263,539.325254,1.370370,2013.000000,638.0,11.000000
std,86.458082,1.610191,108368.878898,2.310661e+05,1215.122637,110.722675,248.462555,12.066970,34.701202,40.500152,1.781462,26.242592,71951.215471,218.487937,4.601195,0.567131,433.610902,2.349599,8.485281,0.0,4.778424
min,0.000000,0.000000,45000.000000,3.221400e+04,1237.110000,0.000000,0.000000,0.000000,1804.000000,2.000000,1.000000,1899.000000,0.000000,0.000000,3.000000,1.000000,108.000000,0.000000,2007.000000,638.0,4.000000
25%,74.500000,2.000000,165000.000000,8.255975e+04,2997.990000,159.800000,245.425000,19.000000,1976.000000,23.500000,2.000000,1973.250000,12565.860000,86.660000,8.000000,2.000000,324.525000,0.000000,2010.000000,638.0,10.000000
50%,149.000000,3.000000,215000.000000,1.352138e+05,3984.060000,210.890000,368.150000,20.000000,2017.000000,32.000000,2.000000,1981.500000,115172.335000,172.200000,10.000000,2.000000,447.270000,1.000000,2013.000000,638.0,10.000000
75%,223.500000,4.000000,297600.000000,2.410000e+05,4762.200000,268.550000,552.617500,20.000000,2022.000000,37.500000,4.000000,2009.500000,166764.500000,367.150000,14.000000,2.000000,611.185000,1.000000,2016.000000,638.0,12.000000
max,298.000000,10.000000,811000.000000,2.390000e+06,6767.860000,981.000000,1293.600000,125.000000,2023.000000,388.000000,8.000000,2022.000000,224261.910000,1016.400000,20.000000,3.000000,3008.540000,10.000000,2019.000000,638.0,24.000000


Analysis of used vocabulary in listing descriptions

In [5]:
# Ideas:
# nltk
# word frequencies
# phrase frequencies


Analysis of price per square meter